In [ ]:
import requests
from abc import ABC, abstractmethod
from typing import List, Optional
import importlib
import backend.adapters.repository as repository
import backend.domain.model as model
import backend.service_layer.service as service
import backend.service_layer.uow as uow
from sec_api import XbrlApi
import pandas as pd
import os
from backend.domain.model import Company, Filing
from backend.service_layer.uow import SqlAlchemyUnitOfWork
from analyze_company_data_completeness import get_sp500_tickers

In [39]:
import yfinance as yf


In [3]:
sp500 = get_sp500_tickers()

In [66]:
uow_instance = uow.UnitOfWork()

In [ ]:
res = service.get_consolidated_income_statements("CMG", uow_instance, "10-K", use_database=False)
res.df 

In [ ]:
res.df

In [ ]:
for ticker in sp500[75:]:
    print(ticker)
    try:
        res = service.get_consolidated_income_statements(ticker, uow_instance, "10-K")
        print(res.analyze_period_coverage()['has_continuous_coverage'])

    except Exception as e:
        print(f'{ticker} failed')
        print(e)


In [ ]:
consolidated = service.get_consolidated_income_statements("GD", uow_instance, "10-K")

In [ ]:
consolidated.df

In [ ]:


with SqlAlchemyUnitOfWork() as uow:
    # Get initial statements
    statements = service.get_consolidated_income_statements("GOOG", uow, "10-K")
    
    uow.stmts.add(statements)

# Start new UoW to verify changes persisted
with SqlAlchemyUnitOfWork() as uow:
    saved_statements = uow.stmts.get("GOOG", "10-K")
    assert saved_statements is not None

In [ ]:
saved_statements.df

In [ ]:
from backend.adapters.repository import PostgresCombinedFinancialStatementsRepository
with SqlAlchemyUnitOfWork() as uow:
    pcfs = PostgresCombinedFinancialStatementsRepository(uow)
    print(pcfs._serialize(statements))

In [ ]:
uow_instance = uow.UnitOfWork()


In [ ]:
combined = service.get_consolidated_income_statements('B ABA', uow_instance, '10-Q')

In [ ]:
for filing in combined.source_filings:
    print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus) 

In [5]:
def remove_zero_decimals(val):
    try:
        num = float(val)
        if num == int(num):
            return int(num)
        else:
            return num
    except (ValueError, TypeError):
        return val

In [6]:
import pandas as pd
pd.options.display.float_format = '{:.10g}'.format
combined.df = combined.df.map(remove_zero_decimals)

In [ ]:
combined.df

In [ ]:
combined.df.apply(combined.is_sparse_row, axis=1)

In [9]:
def convert_to_millions(val):
    try:
        num = float(val)
        
        if abs(num) >= 100_000:
            result = round(num / 1000000, 2)
            # Check if the result has no meaningful decimal part
            if result == int(result):
                return int(result)
            else:
                return result
        else:
            return num
    except (ValueError, TypeError):
        return val

In [10]:
def is_sparse_row(row):
    # Count zero values in the row
    numeric_values = 0
    zero_values = 0
    
    for val in row:
        try:
            num = float(val)
            numeric_values += 1
            if num == 0:
                zero_values += 1
        except (ValueError, TypeError):
            # Skip non-numeric values
            pass
    
    # If no numeric values, don't delete
    if numeric_values == 0:
        return False
        
    # Calculate percentage of zeros
    zero_percentage = zero_values / numeric_values if numeric_values > 0 else 0
    
    # Return True if more than 50% zeros
    return zero_percentage > 0.5

In [11]:
# combined.df = combined.df.applymap(convert_to_millions)


In [12]:
# combined.df = combined.df.loc[[not is_sparse_row(row) for _, row in combined.df.iterrows()]]

In [ ]:
combined.df

In [ ]:
combined.analyze_period_coverage()

In [ ]:
df

In [ ]:
for filing in combined.source_filings:
    try: 
        print(filing.cover_page.document_fiscal_period_focus, filing.cover_page.document_fiscal_year_focus)
    except:
        print(filing.form)

In [ ]:
sorted_columns = sorted(combined.df.columns, key=lambda x: x.split(':')[0])
print(sorted_columns)

In [ ]:
# Get data for first 4 filings
for filing in company.filings[:1]:
    filing.data = uow_instance.sec_filings.get_filing_data(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    filing.filing_url = uow_instance.sec_filings.get_filing_url(
        filing.cik,
        filing.accession_number,
        filing.primary_document
    )
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')


In [ ]:
res = company.select_filings_with_processing_pattern(company.filings,'10-Q')

In [ ]:
for filing in res:
    print(f'Loaded data for filing date {filing.filing_date}, form {filing.form}')